# SECS implementation i Python

Create a figure illustrating Python implementation spherical elementary current system (SECS) analysis on a cubed sphere grid (Figure 3 in paper).

Import relevant modules:

In [ ]:
import lompe
import numpy as np
import matplotlib.pyplot as plt

Re = 6371.2e3    # Earth radius

Set up cubed sphere projection and grid:

In [ ]:
L, W, Lres, Wres = 20000e3, 20000e3, 50e3, 50e3
projection = lompe.cs.CSprojection(position = (0, 90), orientation = 0)
grid = lompe.cs.CSgrid(projection, L, W, Lres, Wres, R = Re+110e3)